In [1]:
!pip install qiskit qiskit-aer --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [2]:
# Quantum Phase Estimation (QPE) using Qiskit 2.x

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt

def qpe_circuit(num_count_qubits, unitary, theta):
    """
    Constructs the Quantum Phase Estimation (QPE) circuit.
    num_count_qubits: number of counting qubits
    unitary: unitary gate (to apply controlled operations)
    theta: phase parameter (for U gate)
    """
    qc = QuantumCircuit(num_count_qubits + 1, num_count_qubits)

    # Step 1: Apply Hadamard gates on counting qubits
    qc.h(range(num_count_qubits))

    # Step 2: Prepare eigenstate (|1>) for the target qubit
    qc.x(num_count_qubits)

    # Step 3: Apply controlled unitary operations
    for qubit in range(num_count_qubits):
        qc.cp(2 * np.pi * theta * (2 ** qubit), qubit, num_count_qubits)

    # Step 4: Apply inverse QFT to counting qubits
    inverse_qft(qc, num_count_qubits)

    # Step 5: Measure counting qubits
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    return qc

In [3]:
def inverse_qft(qc, n):
    """Apply the inverse Quantum Fourier Transform on n qubits."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for k in range(j):
            qc.cp(-np.pi / 2 ** (j - k), k, j)
        qc.h(j)
    return qc

In [4]:
def run_qpe(num_count_qubits=3, theta=0.125):
    """Executes the QPE circuit and visualizes the phase estimation result."""
    simulator = AerSimulator()
    qc = qpe_circuit(num_count_qubits, "U", theta)
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=2048).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    print(qc.draw(output='text'))

In [5]:
if __name__ == "__main__":
    num_count_qubits = 3
    theta = 0.125  # phase value (1/8)
    print(f"Running Quantum Phase Estimation with {num_count_qubits} counting qubits and phase {theta}")
    run_qpe(num_count_qubits, theta)

Running Quantum Phase Estimation with 3 counting qubits and phase 0.125
     ┌───┐                            ┌───┐                                   »
q_0: ┤ H ├─■────────────────────────X─┤ H ├─■──────────────■──────────────────»
     ├───┤ │                        │ └───┘ │P(-π/2) ┌───┐ │                  »
q_1: ┤ H ├─┼────────■───────────────┼───────■────────┤ H ├─┼─────────■────────»
     ├───┤ │        │               │                └───┘ │P(-π/4)  │P(-π/2) »
q_2: ┤ H ├─┼────────┼────────■──────X──────────────────────■─────────■────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π)                                            »
q_3: ┤ X ├─■────────■────────■────────────────────────────────────────────────»
     └───┘                                                                    »
c: 3/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«     ┌─┐           
«q_0: ┤M├───────────
«     

In [9]:
!pip install qiskit==1.1.0 qiskit-aer==0.14.1 --quiet

import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit import transpile
from qiskit_aer import AerSimulator


In [10]:
def run(qc, shots=1024):
    backend = AerSimulator()
    tqc = transpile(qc, backend)
    result = backend.run(tqc, shots=shots).result()
    return result.get_counts()


Task1: Change the Phase Value Try different values of theta (e.g., 0.25, 0.375, 0.5) and see how the measured output changes.

In [11]:
def qpe(theta, n_count=3):
    qc = QuantumCircuit(n_count + 1, n_count)

    # Prepare |1>
    qc.x(n_count)

    # Apply Hadamards on counting qubits
    for q in range(n_count):
        qc.h(q)

    # Apply controlled U^(2^k)
    for q in range(n_count):
        qc.cp(2 * np.pi * theta * (2**q), q, n_count)

    # Inverse QFT
    qc.append(QFT(n_count, inverse=True), range(n_count))

    qc.measure(range(n_count), range(n_count))
    return qc

thetas = [0.25, 0.375, 0.5]
for t in thetas:
    qc = qpe(t)
    counts = run(qc)
    print(f"θ = {t} → Measurement: {counts}")


/tmp/ipython-input-2548803292.py:16: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_count, inverse=True), range(n_count))


θ = 0.25 → Measurement: {'010': 1024}
θ = 0.375 → Measurement: {'011': 1024}
θ = 0.5 → Measurement: {'100': 1024}


Task 2: Increase the Number of Counting Qubits Use 4 or 5 counting qubits for higher precision phase estimation.



In [12]:
for n in [4, 5]:
    print(f"\nUsing {n} counting qubits for θ = 0.375:")
    qc = qpe(0.375, n)
    print(run(qc))



Using 4 counting qubits for θ = 0.375:


/tmp/ipython-input-2548803292.py:16: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_count, inverse=True), range(n_count))


{'0110': 1024}

Using 5 counting qubits for θ = 0.375:
{'01100': 1024}


Task 3: Compare with Theoretical Output Calculate the expected binary representation of the phase and compare with simulation results.

In [13]:
theta = 0.375
n_count = 5

expected_decimal = theta * (2**n_count)
expected_binary = format(int(round(expected_decimal)), f'0{n_count}b')

qc = qpe(theta, n_count)
counts = run(qc)

print("Expected Binary:", expected_binary)
print("Most Frequent Output:", max(counts, key=counts.get))
print("Counts:", counts)


Expected Binary: 01100
Most Frequent Output: 01100
Counts: {'01100': 1024}


/tmp/ipython-input-2548803292.py:16: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_count, inverse=True), range(n_count))


Task 4: Inverse QFT Visualization Add a qc.draw('mpl') command before measurement to visualize the inverse QFT structure.

In [19]:
import matplotlib.pyplot as plt

theta = 0.375
n_count = 4

qc = qpe(theta, n_count)
qc.remove_final_measurements()

qc.draw()




/tmp/ipython-input-2548803292.py:16: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_count, inverse=True), range(n_count))


┌───┐                                    ┌───────┐
q_0: ┤ H ├─■──────────────────────────────────┤0      ├
     ├───┤ │                                  │       │
q_1: ┤ H ├─┼─────────■────────────────────────┤1      ├
     ├───┤ │         │                        │  IQFT │
q_2: ┤ H ├─┼─────────┼─────────■──────────────┤2      ├
     ├───┤ │         │         │              │       │
q_3: ┤ H ├─┼─────────┼─────────┼───────■──────┤3      ├
     ├───┤ │P(3π/4)  │P(3π/2)  │P(3π)  │P(6π) └───────┘
q_4: ┤ X ├─■─────────■─────────■───────■───────────────
     └───┘

Task 5: Noise Simulation Introduce a NoiseModel using Qiskit Aer and observe how it affects accuracy.

In [23]:
from qiskit_aer.noise import NoiseModel, depolarizing_error

# Create a noise model
noise_model = NoiseModel()

# 1-qubit gate noise (H gate)
single_qubit_error = depolarizing_error(0.01, 1)
noise_model.add_all_qubit_quantum_error(single_qubit_error, ['h'])

# 2-qubit gate noise (Controlled-Phase cp gate)
two_qubit_error = depolarizing_error(0.02, 2)
noise_model.add_all_qubit_quantum_error(two_qubit_error, ['cp'])

# Use noisy simulator backend
backend_noisy = AerSimulator(noise_model=noise_model)

# Generate QPE circuit
qc_noisy = qpe(0.375, 4)

# Transpile for noisy backend
tqc_noisy = transpile(qc_noisy, backend_noisy)

# Run!
result_noisy = backend_noisy.run(tqc_noisy, shots=1024).result()
noisy_counts = result_noisy.get_counts()
plot_histogram(noisy_counts)

print("Noisy Output:", noisy_counts)


Noisy Output: {'1111': 7, '0100': 24, '0000': 9, '0010': 11, '1010': 24, '1011': 3, '0011': 3, '0111': 18, '1100': 7, '1000': 23, '0110': 860, '0101': 6, '1110': 29}


/tmp/ipython-input-2548803292.py:16: DeprecationWarning: The class ``qiskit.circuit.library.basis_change.qft.QFT`` is deprecated as of Qiskit 2.1. It will be removed in Qiskit 3.0. ('Use qiskit.circuit.library.QFTGate or qiskit.synthesis.qft.synth_qft_full instead, for access to all previous arguments.',)
  qc.append(QFT(n_count, inverse=True), range(n_count))
